The advection-diffusion equation for tracers are:
$$\kappa\nabla^2\theta = \frac{\partial\theta}{\partial t} + \nabla\cdot({\bf v}\theta)$$

Now, the **left-hand side (dissipation)** is discretized in the following manner. The area integrated flux due to down-gradient [harmonic](https://github.com/MITgcm/MITgcm/blob/master/pkg/generic_advdiff/gad_diff_x.F) + [biharmonic](https://github.com/MITgcm/MITgcm/blob/master/pkg/generic_advdiff/gad_biharm_x.F) diffusion of a tracer is:
$$F^x_{\tt diff} = - A^x \kappa_h \frac{1}{\Delta x_c} \delta_i \theta + \kappa_4 \partial_x \nabla^2 \theta$$
$$F^y_{\tt diff} = - A^y \kappa_h \frac{1}{\Delta y_c} \delta_j \theta + \kappa_4 \partial_y \nabla^2 \theta$$
$$F^r_{\tt diff} = - A^r \kappa_r \frac{1}{\Delta r_c} \delta_k \theta$$
where vertical biharmonic diffusion is not turned on (`DiffKr4` is never called in `package.conf`). The laplacian terms are defined in [`gad_del2.F`](https://github.com/MITgcm/MITgcm/blob/master/pkg/generic_advdiff/gad_del2.F). The [diffusive term](https://github.com/MITgcm/MITgcm/blob/master/pkg/generic_advdiff/gad_calc_rhs.F), therefore, becomes:
$$G_{\tt diff} = \nabla\cdot {\bf F}_{\tt diff}$$

The deep and cosine factors that show up in the discretization are defined [here](https://github.com/MITgcm/MITgcm/blob/master/model/src/set_grid_factors.F), but are all set to unity as the `chaocean` runs set `deepAtmosphere = .False.` and `cosPower = 0.`

For the advection flux on the **right-hand side**, we use scheme No. 33, i.e. [third-order direct space-time with flux limiting (DST3FL)](https://github.com/MITgcm/MITgcm/blob/master/pkg/generic_advdiff/gad_dst3fl_adv_x.F):
$$F_{\tt adv}^x (\theta^{(i)}) = \frac{1}{2}(u+|u|)\bigg(\theta^{(i-1)} + \psi\Big(\frac{\theta^{(i-1)}-\theta^{(i-2)}}{\theta^{(i)}-\theta^{(i-1)}}\Big)(\theta^{(i)}-\theta^{(i-1)})\bigg) + \frac{1}{2}(u-|u|)\bigg(\theta^{(i)} + \psi\Big(\frac{\theta^{(i+1)}-\theta^{(i)}}{\theta^{(i)}-\theta^{(i-1)}}\Big)(\theta^{(i)}-\theta^{(i-1)})\bigg)$$
where the Sweby limiter is
$$\psi(r) = \text{max}\Big[0,\text{min}\big[\text{min}(1, d_0+d_1r], \frac{1-c}{c}r\big]\Big]$$

DST3FL is implemented as [multi-dimensional advection](https://github.com/MITgcm/MITgcm/blob/master/pkg/generic_advdiff/gad_advection.F):
- $\theta^{(i+1/3)} = \theta^{(i)}
     - \Delta t \partial_x \big(F^x(\theta^{(i)}) + \theta^{(i)} \partial_x u\big)$
- $\theta^{(i+2/3)} = \theta^{(i+1/3)}
     - \Delta t \partial_y \big(F^y(\theta^{(i+1/3)}) + \theta^{(i)} \partial_y v\big)$
- $\theta^{(i+3/3)} = \theta^{(i+2/3)}
     - \Delta t \partial_r \big(F^z(\theta^{(i+2/3)}) + \theta^{(i)} \partial_r w\big)$
- $G_\theta = ( \theta^{(i+3/3)} - \theta^{(i)} )/\Delta t$ 

where ${\bf F}(\theta) = {\bf v}\theta$.

Using the 5-day averaged outputs of ${\bf v}$ and $\theta$, however, it is not possible to close the budget as **we are missing the Reynolds-flux like terms**, i.e.
$$\kappa\nabla^2\overline{\theta}^t - \frac{\partial\overline{\theta}^t}{\partial t} - \nabla\cdot(\overline{{\bf v}}^t\overline{\theta}^t) = \nabla\cdot(\overline{{\bf v}'\theta'}^t)$$